In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + input)), negative_slope=0.1)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
   ConvTranspose2d-5            [-1, 256, 8, 8]       1,179,904
       BatchNorm2d-6            [-1, 256, 8, 8]             512
   ConvTranspose2d-7          [-1, 128, 16, 16]         295,040
       BatchNorm2d-8          [-1, 128, 16, 16]             256
   ConvTranspose2d-9            [-1, 2, 32, 32]           2,306
      BatchNorm2d-10            [-1, 2, 32, 32]               4
Total params: 2,662,278
Trainable params: 2,662,278
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 6.78
Params size (MB): 10.16
Estima

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00316335 (0.02706008), PSNR 24.99852180 (19.37332043), SSIM 0.76656020 (0.53827089)
Finished training epoch 0
Validate: MSE 0.00321683 (0.00276962), PSNR 24.92571640 (25.65130158), SSIM 0.70178306 (0.77412582)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00325769 (0.00301425), PSNR 24.87089539 (25.23656455), SSIM 0.74493557 (0.75969361)
Finished training epoch 1
Validate: MSE 0.00311329 (0.00269978), PSNR 25.06780052 (25.76300985), SSIM 0.70362204 (0.77442443)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00261016 (0.00296259), PSNR 25.83333015 (25.30987160), SSIM 0.74282467 (0.75954014)
Finished training epoch 2
Validate: MSE 0.00293308 (0.00263936), PSNR 25.32676125 (25.86053266), SSIM 0.70296729 (0.77219752)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00296092 (0.00288718), PSNR 25.28573227 (25.42378858), SSIM 0.75737017 (0.75818060)
Finished training epoch 3
Validate: MSE 0.00294643 (0.0

Epoch: 30, MSE 0.00239886 (0.00263319), PSNR 26.19994736 (25.81986911), SSIM 0.77860653 (0.76671766)
Finished training epoch 30
Validate: MSE 0.00278245 (0.00235934), PSNR 25.55571938 (26.33651317), SSIM 0.71777618 (0.78357712)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00299975 (0.00263874), PSNR 25.22915268 (25.81389911), SSIM 0.74491119 (0.76685910)
Finished training epoch 31
Validate: MSE 0.00284537 (0.00243124), PSNR 25.45861816 (26.20537571), SSIM 0.71033466 (0.77656503)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00254359 (0.00263045), PSNR 25.94553566 (25.82506389), SSIM 0.77587330 (0.76663451)
Finished training epoch 32
Validate: MSE 0.00260549 (0.00236885), PSNR 25.84110832 (26.31479338), SSIM 0.71814108 (0.78149666)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00231781 (0.00262624), PSNR 26.34922600 (25.83256836), SSIM 0.75288618 (0.76705051)
Finished training epoch 33
Validate: MSE 0.00267117 (0.00239000), PSNR 

Epoch: 60, MSE 0.00196961 (0.00258207), PSNR 27.05618668 (25.90439558), SSIM 0.79561007 (0.76748211)
Finished training epoch 60
Validate: MSE 0.00246174 (0.00252889), PSNR 26.08757210 (26.03568301), SSIM 0.71719414 (0.77381296)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00273043 (0.00257499), PSNR 25.63768578 (25.91718956), SSIM 0.75133026 (0.76750441)
Finished training epoch 61
Validate: MSE 0.00262487 (0.00240405), PSNR 25.80891609 (26.25745755), SSIM 0.72689259 (0.78198341)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00237347 (0.00257894), PSNR 26.24616241 (25.90715561), SSIM 0.77371144 (0.76761840)
Finished training epoch 62
Validate: MSE 0.00281266 (0.00253755), PSNR 25.50882530 (26.02150272), SSIM 0.71549547 (0.77958862)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00309432 (0.00257699), PSNR 25.09434700 (25.91207969), SSIM 0.76405919 (0.76723971)
Finished training epoch 63
Validate: MSE 0.00263697 (0.00230911), PSNR 

Epoch: 90, MSE 0.00300227 (0.00254664), PSNR 25.22549629 (25.96463654), SSIM 0.76793593 (0.76703377)
Finished training epoch 90
Validate: MSE 0.00249166 (0.00239424), PSNR 26.03511620 (26.26899159), SSIM 0.71948016 (0.77593769)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00248406 (0.00255054), PSNR 26.04838371 (25.95781151), SSIM 0.75589257 (0.76702450)
Finished training epoch 91
Validate: MSE 0.00269551 (0.00253082), PSNR 25.69359016 (26.04023162), SSIM 0.71357989 (0.77515783)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00227318 (0.00254209), PSNR 26.43366051 (25.97094298), SSIM 0.77409893 (0.76754890)
Finished training epoch 92
Validate: MSE 0.00256796 (0.00225461), PSNR 25.90410805 (26.52720265), SSIM 0.72144282 (0.78564302)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00256750 (0.00255225), PSNR 25.90489578 (25.95691107), SSIM 0.76430917 (0.76719090)
Finished training epoch 93
Validate: MSE 0.00263046 (0.00226921), PSNR 

Epoch: 120, MSE 0.00241363 (0.00252593), PSNR 26.17329216 (25.99804764), SSIM 0.76260662 (0.76733493)
Finished training epoch 120
Validate: MSE 0.00260812 (0.00240481), PSNR 25.83672905 (26.24926977), SSIM 0.71973407 (0.77737230)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00220881 (0.00252033), PSNR 26.55840683 (26.00959475), SSIM 0.76129591 (0.76734149)
Finished training epoch 121
Validate: MSE 0.00251140 (0.00229834), PSNR 26.00083733 (26.44648908), SSIM 0.71981251 (0.77776020)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00222226 (0.00251574), PSNR 26.53204346 (26.01990122), SSIM 0.74170464 (0.76708885)
Finished training epoch 122
Validate: MSE 0.00251224 (0.00225435), PSNR 25.99938202 (26.52910782), SSIM 0.72215259 (0.77825822)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00234221 (0.00253339), PSNR 26.30375099 (25.98628500), SSIM 0.76917952 (0.76699467)
Finished training epoch 123
Validate: MSE 0.00258089 (0.00228

Validate: MSE 0.00233232 (0.00224993), PSNR 26.32212448 (26.53155346), SSIM 0.72652507 (0.78114483)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00199119 (0.00251716), PSNR 27.00887489 (26.01666084), SSIM 0.78286469 (0.76679836)
Finished training epoch 150
Validate: MSE 0.00260678 (0.00229111), PSNR 25.83895874 (26.46076553), SSIM 0.72061211 (0.78473316)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00246431 (0.00250614), PSNR 26.08304405 (26.03393156), SSIM 0.75439799 (0.76707457)
Finished training epoch 151
Validate: MSE 0.00243687 (0.00223767), PSNR 26.13167381 (26.56069815), SSIM 0.71972907 (0.78030121)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00268136 (0.00250432), PSNR 25.71645164 (26.03555912), SSIM 0.75874478 (0.76718908)
Finished training epoch 152
Validate: MSE 0.00244086 (0.00229509), PSNR 26.12457275 (26.44957122), SSIM 0.72110212 (0.77698469)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

Epoch: 179, MSE 0.00188633 (0.00249886), PSNR 27.24381256 (26.04647411), SSIM 0.77810878 (0.76677729)
Finished training epoch 179
Validate: MSE 0.00265321 (0.00232020), PSNR 25.76228142 (26.40586967), SSIM 0.72363663 (0.78202647)
Finished validation.
Starting training epoch 180
Epoch: 180, MSE 0.00216611 (0.00249631), PSNR 26.64318275 (26.05038695), SSIM 0.75998873 (0.76690628)
Finished training epoch 180
Validate: MSE 0.00250383 (0.00224965), PSNR 26.01394272 (26.53431525), SSIM 0.73030245 (0.78275024)
Finished validation.
Starting training epoch 181
Epoch: 181, MSE 0.00174166 (0.00248108), PSNR 27.59037399 (26.07595586), SSIM 0.78971660 (0.76728412)
Finished training epoch 181
Validate: MSE 0.00259081 (0.00234129), PSNR 25.86563683 (26.35484727), SSIM 0.71917641 (0.77343521)
Finished validation.
Starting training epoch 182
Epoch: 182, MSE 0.00288182 (0.00250771), PSNR 25.40333176 (26.03188876), SSIM 0.76446736 (0.76657085)
Finished training epoch 182
Validate: MSE 0.00249182 (0.00231

Validate: MSE 0.00246350 (0.00223941), PSNR 26.08446884 (26.55656784), SSIM 0.72554731 (0.78158172)
Finished validation.
Starting training epoch 209
Epoch: 209, MSE 0.00240776 (0.00248803), PSNR 26.18386650 (26.06781050), SSIM 0.74620354 (0.76650352)
Finished training epoch 209
Validate: MSE 0.00242807 (0.00229143), PSNR 26.14738846 (26.46005745), SSIM 0.73067534 (0.78321672)
Finished validation.
Starting training epoch 210
Epoch: 210, MSE 0.00255952 (0.00249068), PSNR 25.91842079 (26.06098356), SSIM 0.77031624 (0.76646641)
Finished training epoch 210
Validate: MSE 0.00247726 (0.00225437), PSNR 26.06028175 (26.52748214), SSIM 0.72066802 (0.77719866)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00247805 (0.00225417), PSNR 26.05890083 (26.52819202), SSIM 0.72174442 (0.77698171)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()